In [1]:
import numpy as np
import struct

In [2]:
def float_to_bin(num):
    return bin(struct.unpack('!I', struct.pack('!f', num))[0])[2:].zfill(32)


def bin_to_float(binary):
    return struct.unpack('!f', struct.pack('!I', int(binary, 2)))[0]


In [3]:
class ExtendableList(list):
    def __setitem__(self, key, value):
        if isinstance(key, slice):
            if self.__len__() < key.start + 1:
                self.extend([None] * (key.start + 1 - self.__len__()))
            if self.__len__() < key.stop + 1:
                self.extend([None] * (key.stop - self.__len__()))
        else:
            while self.__len__() < key + 1:
                self.extend([None] * (key + 1 - self.__len__()))
        super().__setitem__(key, value)


In [9]:
class Task:
    id = 1

    def __init__(self, resource_list: np.ndarray, costs_list: np.ndarray):
        if len(resource_list) != len(costs_list):
            raise Exception('Resources list and costs list must be the same length')
        self.resource_list = resource_list
        self.costs_list = costs_list
        self.total_cost = np.sum(costs_list)
        self.total_steps = len(self.costs_list)
        self.is_finished = False

        self.step = 0
        self.last_step_finished = 0
        self.next_idle_time = 0
        self.id = Task.id
        Task.id += 1

    def validate_args(self, res: int, t: int):
        if t <= self.next_idle_time:
            raise Exception(
                f'Last step is not done yet. Passed time is {t}, and current process ends at time={self.next_idle_time}')

        if res != self.resource_list[self.step]:
            raise Exception(
                f'Wrong type of resource. Passed resource is {res} and next needed resource is {self.resource_list[self.step]}')

    def get_next_resource(self, res: int, t: int):
        self.validate_args(res, t)

        return self.resource_list[self.step]

    def get_value_state(self, res: int, t: int):
        self.validate_args(res, t)

        return self.total_cost, self.total_steps - self.step, self.costs_list[self.step], t - self.next_idle_time

    def execute_next_step(self, res: int, t: int):
        self.validate_args(res, t)

        self.next_idle_time = t + self.costs_list[self.step] + 1
        self.step += 1
        if self.step >= self.total_steps:
            self.is_finished = True
        return self.costs_list[self.step]

    def is_ready(self, res: int, t: int) -> bool:
        return self.next_idle_time <= t and self.resource_list[self.step] == res


In [5]:
def create_maufacturing_queues(tasks: list, alpha: float, beta: float, gamma: float, delta: float):
    if len(tasks) == 0:
        raise Exception('There are no tasks')

    resources_queue_list = [ExtendableList() for _ in range(np.unique(np.concatenate([task.resource_list for task in tasks]))[-1]+1)]
    tasks_left = [task for task in tasks if not task.is_finished]
    while len(tasks_left):
        pass

    return resources_queue_list


In [5]:
x = np.genfromtxt('GA_task.csv', delimiter=";", dtype=int)

In [10]:
tasks  = [Task(x[:, i], x[:, i+1]) for i in range(0, x.shape[1], 2)]

In [38]:
test = ExtendableList()
start = 0
duration = 2
stop = start + duration
test[start:stop] = [2 for _ in range(start, stop)]

In [39]:
test

[2, 2]

In [35]:
from typing import Callable
import random


def generate_genotype(n_features: int) -> str:
    return ''.join([random.choice(['0', '1']) for _ in range(n_features * 32)])


class UnitFloat:
    def __init__(self, genotype: str = None, n_features: int = 4, cost_function: Callable[[list], int] = None):
        self.genotype = genotype
        if genotype is None:
            self.genotype = generate_genotype(n_features)

        self.n_features = n_features
        self.cost_function = cost_function

    @property
    def fitness(self):
        return self.cost_function(self.resources_queue_list)

    @property
    def phenotype(self):
        return [bin_to_float(self.genotype[i*32:i*32+32]) for i in range(self.n_features)]

test = UnitFloat()
print(test.phenotype)

[828427.9375, -8.99690055847168, -8.155193851642485e+19, 5.819401849294081e-08]
